# Python Program for Calculating Call or Put Buying Total

In [1]:
#importing Imp Libraries 

import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
import datetime, time, math
from selenium.webdriver.support.ui import Select
start_time = time.time()

# Create a session and load the page
chromedriver_path = 'chromedriver.exe' # Change it to your own chromedriver path!
driver = webdriver.Chrome(executable_path=chromedriver_path)
#driver.get('https://www1.nseindia.com/live_market/dynaContent/live_watch/option_chain/optionKeys.jsp')
driver.get('https://www.nseindia.com/option-chain')

# Wait for the page to fully load
driver.implicitly_wait(5)

# Parse HTML code and grab tables with Beautiful Soup
soup = BeautifulSoup(driver.page_source, 'html.parser')

tables = soup.find_all('table')

# Read tables with Pandas read_html()
dfs = pd.read_html(str(tables))

print(f'Total tables: {len(dfs)}')
print(dfs[0])

Total tables: 1
Empty DataFrame
Columns: [(CALLS, Unnamed: 0_level_1), (CALLS, OI), (CALLS, Chng in OI), (CALLS, Volume), (CALLS, IV), (CALLS, LTP), (CALLS, Chng), (CALLS, Bid Qty), (CALLS, Bid Price), (CALLS, Ask Price), (CALLS, Ask Qty), (Unnamed: 11_level_0, Strike Price), (PUTS, Bid Qty), (PUTS, Bid Price), (PUTS, Ask Price), (PUTS, Ask Qty), (PUTS, Chng), (PUTS, LTP), (PUTS, IV), (PUTS, Volume), (PUTS, Chng in OI), (PUTS, OI), (PUTS, Unnamed: 22_level_1)]
Index: []

[0 rows x 23 columns]


In [2]:
table=driver.find_element_by_xpath('//*[@id="optionChainTable-indices"]')


In [3]:
table.text

'CALLS PUTS\nOI\nCHNG IN OI\nVOLUME\nIV\nLTP\nCHNG\nBID QTY\nBID PRICE\nASK PRICE\nASK QTY\nSTRIKE PRICE\nBID QTY\nBID PRICE\nASK PRICE\nASK QTY\nCHNG\nLTP\nIV\nVOLUME\nCHNG IN OI\nOI\n9 8 18 - 2,000.00 -92.75 1,050 2,021.35 2,066.00 975 12,800.00 13,425 1.10 1.30 8,400 -0.30 1.15 44.73 13,448 2,251 6,006\n- - - - - - 2,625 1,940.40 2,109.15 2,625 12,850.00 675 1.35 1.50 600 -18.45 1.35 44.45 652 21 21\n- - - - - - 2,700 1,897.30 2,025.25 75 12,900.00 975 1.25 1.40 6,450 -0.25 1.30 43.20 3,968 638 1,043\n- - - - - - 2,625 1,842.35 2,007.20 2,625 12,950.00 975 1.15 1.70 150 -24.35 1.65 43.29 66 21 21\n68 - 7 - 1,841.15 -123.85 1,500 1,841.10 1,854.55 150 13,000.00 75 1.65 1.75 750 0.05 1.75 42.50 31,143 7,088 14,735\n- - - - - - 2,700 1,748.25 1,886.45 75 13,050.00 75 1.35 2.25 75 -0.35 1.80 41.55 130 41 77\n- - 2 - 1,751.50 160.00 75 1,720.70 1,793.60 2,625 13,100.00 2,700 1.80 1.95 375 - 1.95 40.85 6,927 1,705 2,312\n- - - - - - 2,700 1,649.50 1,769.05 75 13,150.00 75 1.45 2.20 150 -0

In [5]:
#create an empty df
df = pd.DataFrame([table.text]) 

In [16]:
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       1 non-null      object
dtypes: object(1)
memory usage: 136.0+ bytes


In [24]:
df.reset_index(level=0, drop=True)

,0
0,CALLS PUTS\nOI\nCHNG IN OI\nVOLUME\nIV\nLTP\nC...


In [32]:
df=df.dropna(axis=1,thresh=len(df)).reset_index()

In [33]:
df

,index,0
0,0,CALLS PUTS\nOI\nCHNG IN OI\nVOLUME\nIV\nLTP\nC...


In [29]:
df.columns = df.columns.droplevel()

ValueError: Cannot remove 1 levels from an index with 1 levels: at least one level must be left.

In [ ]:
#saving df into csv
#n=str(datetime.datetime.now().strftime('%Y-%m-%d %H-%M-%S'))
#name='NSE'+n+'.csv'
#df.to_csv(name) 

In [8]:
#getting strick price from web page
strick_price = driver.find_element_by_xpath('//*[@id="equity_underlyingVal"]')
strick_price.text

'NIFTY 14,834.85'

In [11]:
LTP=(strick_price.text[6:])
print(type(LTP))
LTP.strip()
LTP = LTP.replace(',', '')
LTP=int(float(LTP))
print(LTP)

<class 'str'>
14834


In [12]:
#logic for strick price conversion of multiple of 150 -50
f3digit=LTP//100
print("First 3 digit",f3digit)

num=LTP//10
last=num%10
print("Last: ",last)
if last>=5:
    f3digit=(f3digit+1)*100
else:
    f3digit=(f3digit)*100
    f3digit+=50
    
print(f3digit)

First 3 digit 148
Last:  3
14850


In [ ]:
#drop unneccessary columns from df
df.drop(df.columns[[0,1,4,5,6,7,8,9,10,12,13,14,15,16,17,18,19,22]], axis = 1, inplace = True) 
#df.head() 

In [ ]:
#df.columns

In [ ]:
df.columns = df.columns.droplevel() #it drops multilevel indexing

In [ ]:
#df.head()

In [ ]:
#giving proper names to our new columns
df.columns = ['Call_Chng_in_OI', 'Call_Volume', 'Strike Price', 'Put_Chng_in_OI', 'Put_OI']

In [ ]:
 #it gives index of matched column value in int form
i=df.loc[df['Strike Price']==f3digit].index.values.astype(int)[0]
#i

In [ ]:
start=i-5
df2=df.iloc[start:start+11,]

In [ ]:
#df2.columns

In [ ]:
#df2.head()

In [ ]:
#df2.info()

In [ ]:
#saving df into csv
#n=str(datetime.datetime.now().strftime('%Y-%m-%d %H-%M-%S'))
#name='ST_NSE'+n+'.csv'
#df2.to_csv(name) 

In [ ]:
try:
    c_col=(df2['Call_Chng_in_OI'].astype(str).astype(int)).sum()
   # print(c_col)
    p_col=(df2['Put_Chng_in_OI'].astype(str).astype(int)).sum()
  #  print(p_col)
except Exception as e:
    c_col=0
    p_col=0
    print('Ecxeption')
    

In [ ]:
if c_col>=p_col:
    signal="Buy PE"
    print("Buy a PE Call")
else:
    signal="Buy CE"
    print("Buy a CE Call")

In [ ]:
df4 = pd.DataFrame(columns=['Date_Time', 'Sum_Call_OI', 'Strike_price', \
                            'Sum_Put_OI', 'Difference','Signal'])

In [ ]:
date_time=str(datetime.datetime.now().strftime('%H-%M-%S'))

difference = p_col-c_col
df4 = df4.append({'Date_Time':date_time,\
                      'Sum_Call_OI': c_col,\
                      'Strike_price':f3digit,\
                      'Sum_Put_OI': p_col,\
                      'Difference': difference,\
                      'Signal':signal},\
                     ignore_index=True)



In [ ]:
#saving finaL df4 into csv
date_time=str(datetime.datetime.now().strftime('%Y-%m-%d'))
name_nifty='NIFTY_OI_NSE'+date_time+'.csv'
df4.to_csv(name_nifty) 

In [ ]:
driver.close()

#Save Every points Text File
#File Name= Output

n=str(datetime.datetime.now().strftime('%Y-%m-%d'))
#name='NSE'+n+'.csv'
fname='output'+n+'.txt'
stltp='\n'+"Strick Price LTP: "+str(LTP)+'\n'
stltp_="SP on Data: "+str(f3digit)+'\n'
CE_PE_DATA="CE Data: "+str(c_col)+" PE DATA: "+str(p_col)+'\n'
signal1='Signal:'+signal+'\n'
diff=p_col-c_col
diff=str(diff)+'\n'

sec=((time.time() - start_time))
time_req="Required Seconds for Exe:" + str(math.trunc(sec))+'\n'
with open(fname,'a') as f:
    f.write(datetime.datetime.now().strftime('%H:%M:%S'))
    f.write(stltp)
    f.write(stltp_)
    f.write(CE_PE_DATA)
    f.write(signal1)
    f.write(diff)
    f.write(time_req)
    f.write("-------------++++++++--------------\n")
    del signal1
print("Data Updated Succesfully...!")
print("--- %s seconds ---" % math.trunc(time.time() - start_time))
time.sleep(15)
driver.close()

# SGX Nifty Data

# Bank Nifty Last five days Open Close Data

In [ ]:

import matplotlib.pyplot as plt

%matplotlib inline

#df=pd.read_csv(name_nifty)
x=df4['Date_Time']
y=df4['Difference']
plt.plot(x,y,color='blue',marker='D',markersize=6,linestyle="dashdot",label="diff")
#plt.plot(x,df['Strike_price'],label="ST")
plt.xticks(rotation=80)
plt.xlabel("Time")
plt.ylabel("Difference")
plt.title("OI CE & PE Live Difference. ")
plt.legend(loc="best",shadow=True,fontsize="small")#Adding shadow to legend
#plt.savefig('chart.png')

In [ ]:
df4.tail()

In [ ]:
ax = plt.gca() # gca stands for 'get current axis'
 
df4.plot(kind='line',x= 'Date_Time',y='Difference',figsize=(18,3),grid=True,ax=ax,rot=45)
df4.plot(kind='line',x= 'Date_Time',y='Sum_Call_OI',figsize=(18,3),grid=True,ax=ax,rot=45)
df4.plot(kind='line',x= 'Date_Time',y='Sum_Put_OI',figsize=(18,3),grid=True,ax=ax,rot=45)
plt.savefig('output.png')

In [ ]:
class Line:
    
    def __init__(self,coor1,coor2):
        self.coor1=coor1
        self.coor2=coor2
      
    
    def distance(self):
        pass
    
    def slope(self):
        pass
# EXAMPLE OUTPUT

coordinate1 = (3,2)
coordinate2 = (8,10)

li = Line(coordinate1,coordinate2)

In [ ]:
def fizzBuzz(n):
    if n%3==0 and n%5==0:
        print('FizzBuzz')
    elif n%3==0:
        print('Fizz')
    elif n%5==0:
        print('Buzz')
    else:
        print(n)

if __name__ == '__main__':
    n = int(input().strip())

    fizzBuzz(n)